In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import cluster
from sklearn import svm
from sklearn import tree
from sklearn import model_selection

In [2]:
from sklearn import model_selection as ms

In [3]:
200*.8/5

32.0

In [4]:
iris_df = sns.load_dataset('iris')
iris_X = iris_df.drop(columns='species')
iris_y = iris_df['species']

iris_X_train, iris_X_test, iris_y_train, iris_y_test = model_selection.train_test_split(iris_X,iris_y, test_size =.2)

In [5]:
dt = tree.DecisionTreeClassifier()
params_dt = {'criterion':['gini','entropy'],'max_depth':[2,3,4],
       'min_samples_leaf':list(range(2,20,2))}

In [6]:
dt_opt = model_selection.GridSearchCV(dt, params_dt)
dt_opt.fit(iris_X_train,iris_y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4],
                         'min_samples_leaf': [2, 4, 6, 8, 10, 12, 14, 16, 18]})

In [7]:
dt_cv_df = pd.DataFrame(dt_opt.cv_results_)

In [8]:
dt_cv_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002363,0.000468,0.001538,0.000082,gini,2,2,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
1,0.002119,0.000053,0.001504,0.000027,gini,2,4,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
2,0.002206,0.000319,0.001526,0.000076,gini,2,6,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
3,0.002021,0.000129,0.001542,0.000054,gini,2,8,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
4,0.002058,0.000063,0.001542,0.000062,gini,2,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
5,0.002004,0.000068,0.001579,0.000081,gini,2,12,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
6,0.001985,0.000053,0.001577,0.000093,gini,2,14,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
7,0.001997,0.000070,0.001564,0.000067,gini,2,16,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
8,0.002006,0.000081,0.001594,0.000077,gini,2,18,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.916667,0.958333,0.958333,0.958333,0.958333,0.950000,0.016667,1
9,0.002185,0.000225,0.001743,0.000527,gini,3,2,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.916667,0.916667,0.958333,0.916667,0.958333,0.933333,0.020412,51


In [9]:
type(dt_opt.best_estimator_)

sklearn.tree._classes.DecisionTreeClassifier

In [10]:
dt_opt.best_params_

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

In [11]:
svmclf = svm.SVC()
param_grid_svm = {'kernel':['linear','rbf'], 'C':[.5, 1, 10]}
svm_opt = model_selection.GridSearchCV(svmclf,param_grid_svm)

In [12]:
svm_opt.fit(iris_X_train,iris_y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.5, 1, 10], 'kernel': ['linear', 'rbf']})

In [13]:
svm_opt.best_params_

{'C': 0.5, 'kernel': 'linear'}

In [14]:
svm_opt.score(iris_X_test,iris_y_test), dt_opt.score(iris_X_test,iris_y_test)

(0.9666666666666667, 0.9)